In [37]:
pip install yfinance pandas numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import yfinance as yf
import pandas as pd


In [39]:
import yfinance as yf
import pandas as pd

# Download data
data = yf.download("^NSEI", start="2022-01-01", end="2024-12-31", interval="1d")
data = data.dropna()
data.reset_index(inplace=True)

# Fix columns if MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

print(data.columns.tolist())  # Check column names

df = data.copy()


[*********************100%***********************]  1 of 1 completed

['Date', 'Close', 'High', 'Low', 'Open', 'Volume']


In [40]:
# Bollinger Bands
def bollinger_bands(df, window=20, num_std=2):
    df['BB_Middle'] = df['Close'].rolling(window).mean()
    df['BB_Std'] = df['Close'].rolling(window).std()
    df['BB_Upper'] = df['BB_Middle'] + num_std * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - num_std * df['BB_Std']
    return df

# MACD
def macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df

# Stochastic Oscillator
def stochastic_oscillator(df, k_window=14, d_window=3):
    df['Low_Min'] = df['Low'].rolling(window=k_window).min()
    df['High_Max'] = df['High'].rolling(window=k_window).max()
    df['%K'] = 100 * (df['Close'] - df['Low_Min']) / (df['High_Max'] - df['Low_Min'])
    df['%D'] = df['%K'].rolling(window=d_window).mean()
    return df

# VWAP
def vwap(df):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['Cumulative_TP_Volume'] = (df['TP'] * df['Volume']).cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df

# RSI
def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# ATR
def atr(df, window=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window).mean()
    return df


In [41]:
print(df.columns.tolist())


['Date', 'Close', 'High', 'Low', 'Open', 'Volume']


In [43]:
df = bollinger_bands(df)
df = macd(df)
df = stochastic_oscillator(df)
df = vwap(df)
df = rsi(df)
df = atr(df)


In [44]:
df.to_csv("nifty50_with_indicators.csv", index=False)
print("Dataset with indicators saved as nifty50_with_indicators.csv")


Dataset with indicators saved as nifty50_with_indicators.csv


In [45]:
import pandas as pd

# Load the processed dataset
df = pd.read_csv("nifty50_with_indicators.csv")
print(df.columns.tolist())  # Check columns to ensure data loaded correctly


['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'BB_Middle', 'BB_Std', 'BB_Upper', 'BB_Lower', 'EMA_fast', 'EMA_slow', 'MACD', 'MACD_Signal', 'Low_Min', 'High_Max', '%K', '%D', 'TP', 'Cumulative_TP_Volume', 'Cumulative_Volume', 'VWAP', 'RSI', 'H-L', 'H-PC', 'L-PC', 'TR', 'ATR']


In [46]:
def bollinger_band_signal(df):
    df['BB_Signal'] = 0
    df.loc[df['Close'] < df['BB_Lower'], 'BB_Signal'] = 1
    df.loc[df['Close'] > df['BB_Upper'], 'BB_Signal'] = -1
    return df

def macd_signal(df):
    df['MACD_Signal_Gen'] = 0
    df.loc[(df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)), 'MACD_Signal_Gen'] = 1
    df.loc[(df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)), 'MACD_Signal_Gen'] = -1
    return df

def stochastic_signal(df):
    df['Stoch_Signal'] = 0
    df.loc[(df['%K'] > 20) & (df['%K'].shift(1) <= 20), 'Stoch_Signal'] = 1
    df.loc[(df['%K'] < 80) & (df['%K'].shift(1) >= 80), 'Stoch_Signal'] = -1
    return df

def vwap_signal(df):
    df['VWAP_Signal'] = 0
    df.loc[(df['Close'] > df['VWAP']) & (df['Close'].shift(1) <= df['VWAP'].shift(1)), 'VWAP_Signal'] = 1
    df.loc[(df['Close'] < df['VWAP']) & (df['Close'].shift(1) >= df['VWAP'].shift(1)), 'VWAP_Signal'] = -1
    return df

def rsi_signal(df):
    df['RSI_Signal'] = 0
    df.loc[(df['RSI'] > 30) & (df['RSI'].shift(1) <= 30), 'RSI_Signal'] = 1
    df.loc[(df['RSI'] < 70) & (df['RSI'].shift(1) >= 70), 'RSI_Signal'] = -1
    return df

def atr_signal(df, multiplier=1.5):
    df['ATR_Signal'] = 0
    df.loc[df['TR'] > multiplier * df['ATR'], 'ATR_Signal'] = 1
    df.loc[df['TR'] < (1/multiplier) * df['ATR'], 'ATR_Signal'] = -1
    return df


In [47]:
df = bollinger_band_signal(df)
df = macd_signal(df)
df = stochastic_signal(df)
df = vwap_signal(df)
df = rsi_signal(df)
df = atr_signal(df)


In [48]:
print("Bollinger Bands: Buy =", (df['BB_Signal'] == 1).sum(), ", Sell =", (df['BB_Signal'] == -1).sum())
print("MACD: Buy =", (df['MACD_Signal_Gen'] == 1).sum(), ", Sell =", (df['MACD_Signal_Gen'] == -1).sum())
print("Stochastic Oscillator: Buy =", (df['Stoch_Signal'] == 1).sum(), ", Sell =", (df['Stoch_Signal'] == -1).sum())
print("VWAP: Buy =", (df['VWAP_Signal'] == 1).sum(), ", Sell =", (df['VWAP_Signal'] == -1).sum())
print("RSI: Buy =", (df['RSI_Signal'] == 1).sum(), ", Sell =", (df['RSI_Signal'] == -1).sum())
print("ATR: Buy =", (df['ATR_Signal'] == 1).sum(), ", Sell =", (df['ATR_Signal'] == -1).sum())


Bollinger Bands: Buy = 28 , Sell = 38
MACD: Buy = 28 , Sell = 28
Stochastic Oscillator: Buy = 47 , Sell = 49
VWAP: Buy = 9 , Sell = 9
RSI: Buy = 20 , Sell = 28
ATR: Buy = 91 , Sell = 149


In [49]:
df.tail(10)  # See the last 10 rows with all signals


,Date,Close,High,Low,Open,Volume,BB_Middle,BB_Std,BB_Upper,BB_Lower,...,H-PC,L-PC,TR,ATR,BB_Signal,MACD_Signal_Gen,Stoch_Signal,VWAP_Signal,RSI_Signal,ATR_Signal
728,2024-12-16,24668.250000,24781.250000,24601.750000,24753.400391,187600,24270.447754,438.649612,25147.746977,23393.148531,...,12.949219,166.550781,179.500000,263.706892,0,0,0,0,0,0
729,2024-12-17,24336.000000,24624.099609,24303.449219,24584.800781,264900,24314.557715,394.323351,25103.204417,23525.911013,...,44.150391,364.800781,364.800781,274.842634,0,0,-1,0,0,0
730,2024-12-18,24198.849609,24394.449219,24149.849609,24297.949219,235300,24348.575195,348.746922,25046.069040,23651.081351,...,58.449219,186.150391,244.599609,258.571150,0,0,0,0,0,0
731,2024-12-19,23951.699219,24004.900391,23870.300781,23877.150391,271100,24378.665137,276.530667,24931.726470,23825.603803,...,193.949219,328.548828,328.548828,262.446150,0,-1,0,0,0,0
732,2024-12-20,23587.500000,24065.800781,23537.349609,23960.699219,442700,24362.677637,312.166708,24987.011053,23738.344220,...,114.101562,414.349609,528.451172,279.260603,1,0,0,0,0,1
733,2024-12-23,23753.449219,23869.550781,23647.199219,23738.199219,189800,24339.255078,339.650109,25018.555295,23659.954861,...,282.050781,59.699219,282.050781,284.742885,0,0,0,0,0,0
734,2024-12-24,23727.650391,23867.650391,23685.150391,23769.099609,177700,24315.912598,365.202940,25046.318478,23585.506717,...,114.201172,68.298828,182.500000,283.000140,0,0,0,0,0,-1
735,2024-12-26,23750.199219,23854.500000,23653.599609,23775.800781,177700,24289.677539,386.528042,25062.733624,23516.621454,...,126.849609,74.050781,200.900391,257.193080,0,0,0,0,0,0
736,2024-12-27,23813.400391,23938.849609,23800.599609,23801.400391,176800,24284.640039,392.293283,25069.226605,23500.053473,...,188.650391,50.400391,188.650391,261.343052,0,0,1,0,0,0
737,2024-12-30,23644.900391,23915.349609,23599.300781,23796.900391,364900,24260.330078,416.619165,25093.568409,23427.091748,...,101.949219,214.099609,316.048828,274.993025,0,0,0,0,0,0
